### Importação das bibliotecas
Nesta etapa, importamos as bibliotecas e pacotes necessários para o desenvolvimento do projeto. As principais bibliotecas usadas são:

- pandas: Para manipulação e análise de dados, fornecendo estrutura de dados flexível e eficiente.
- matplotlib.pyplot e seaborn: Para visualização gráfica, facilitando a interpretação das características do dataset e dos resultados dos modelos.
- LabelEncoder: Do sklearn, para converter colunas categóricas em numéricas, facilitando a aplicação de algoritmos de machine learning.
- RandomForestClassifier: Um dos principais modelos de classificação utilizado para predizer a popularidade das músicas.
- accuracy_score, confusion_matrix, classification_report: Ferramentas para avaliar a performance do modelo.
- train_test_split: Para dividir os dados em conjuntos de treino e teste, garantindo uma avaliação justa.
- RandomizedSearchCV: Para a otimização de hiperparâmetros de maneira mais eficiente, testando diferentes combinações.

In [218]:
# Importa as bibliotecas necessárias

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, RandomizedSearchCV


### Carregamento dos dados
Nesta etapa, os datasets de treino e teste foram carregados utilizando a função pd.read_csv(). O dataset de treino (df_train) será utilizado para ajustar o modelo de machine learning, enquanto o dataset de teste (df_test) será usado para fazer previsões e avaliar o modelo final.

Além disso, uma cópia do dataset de teste foi feita (df_test_original) para preservar a integridade dos dados originais, permitindo comparações futuras e uma possível análise posterior sem alterações no conjunto de teste original.

In [219]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_test_original = df_test

### Exploração Inicial dos Dados

Aqui, utilizamos o método df_train.info() para obter uma visão geral da estrutura do dataset de treino. Essa função fornece informações importantes, como:

- O número total de entradas (linhas) e colunas.
- O tipo de dado de cada coluna (por exemplo, inteiro, float, ou objeto).
- O número de valores não nulos presentes em cada coluna.

Essa análise inicial ajuda a identificar possíveis problemas, como dados ausentes, e permite uma compreensão básica das variáveis que estão presentes no dataset, orientando as etapas subsequentes de limpeza e transformação dos dados.

In [ ]:
# Informa dados sobre a tabela df_train

df_train.info()

### Exploração Inicial dos Dados 2.0
A função df_train.describe() fornece uma visão geral estatística das colunas numéricas do dataset. Essa função retorna métricas como:

- Contagem (count): número de valores não nulos em cada coluna.
- Média (mean): valor médio de cada coluna.
- Desvio padrão (std): medida de dispersão dos dados em relação à média.
- Valores mínimos e máximos (min/max): extremos da distribuição dos dados.
- Quartis (25%, 50%, 75%): valores que dividem os dados em quartis, úteis para detectar a dispersão e a distribuição dos dados.

Essa análise estatística preliminar permite identificar possíveis outliers, entender a distribuição dos dados e ter uma visão mais detalhada de como cada variável se comporta no dataset.

In [ ]:
# Descreve os dados da tabela df_train

df_train.describe()

### Detecção de Valores Nulos
Nessa etapa, foi verificada a presença de valores ausentes (nulos) no dataset de treinamento utilizando df_train.isna().sum(). Nenhuma das colunas apresentou valores nulos, o que é um ótimo indicativo para a continuidade do processo de modelagem, já que não será necessário lidar com técnicas de imputação ou remoção de valores ausentes.

In [220]:
# Verifica valores nulos em todas as colunas
nulos = df_train.isna().sum()

# Repetição para conferir a contagem de nulos
for column in nulos:
    if column > 0:
        print(column)

### Separação de colunas categóricas e numéricas

Nesta etapa, as colunas do dataset foram separadas em categóricas e numéricas para facilitar o processamento subsequente. Colunas do tipo bool e object foram identificadas como categóricas, tanto no conjunto de treino quanto no de teste. Essas colunas foram armazenadas em listas separadas para serem manipuladas de forma específica.

- cat_columns: Armazena as colunas categóricas do dataset de treino.
- cat_columns_test: Armazena as colunas categóricas do dataset de teste.

O dataset foi dividido em dois: df_train_cat e df_test_cat (contendo as colunas categóricas) e df_train_num (contendo as colunas numéricas). Isso permitirá que cada tipo de dado seja tratado de forma adequada em etapas futuras, como codificação de variáveis categóricas e normalização das numéricas.

In [221]:
# Inicia valores de listas
cat_columns = []
cat_columns_test = []

# Repetição para associar a lista as colunas categoricas e numéricas do df_train
for column in df_train:
    if df_train[column].dtype == 'bool':
        cat_columns.append(column)

    if df_train[column].dtype == 'object':
        cat_columns.append(column)

# Repetição para associar a lista as colunas categoricas e numéricas do df_test
for column in df_test:
    if df_train[column].dtype == 'bool':
        cat_columns_test.append(column)

    if df_train[column].dtype == 'object':
        cat_columns_test.append(column)
     

# Associa as variáveis (dataframes) os valores das colunas categóricas e numéricas
df_train_cat = df_train[cat_columns]
df_train_num = df_train.drop(columns=cat_columns)

df_test_cat = df_test[cat_columns_test]

### Exploração de Outliers
Nesta etapa, exploramos a presença de outliers nas features numéricas usando gráficos de boxplot. Cada coluna numérica do conjunto de dados foi visualizada por meio de um boxplot, o que permite identificar valores atípicos que estão fora do intervalo interquartil (IQR).

Para isso, foi criada uma grade de subplots de 5x3, onde cada gráfico exibe a distribuição de uma variável numérica específica. Essa análise visual é importante para entender a dispersão e a presença de possíveis outliers em cada feature, o que pode impactar o desempenho do modelo preditivo.

Após a análise, será possível decidir se será necessário aplicar técnicas de tratamento de outliers, como remoção ou transformação dos dados.

In [ ]:

# Criando boxplots para cada uma das colunas
fig, axes = plt.subplots(5, 3, figsize=(15, 15))
axes = axes.flatten()

# Iterando sobre as colunas do DataFrame e plotando os boxplots
for i, col in enumerate(df_train_num.columns):
    axes[i].boxplot(df_train_num[col])
    axes[i].set_title(f'Boxplot de {col}')
    axes[i].set_ylabel(col)

# Ajustando o layout para não sobrepor os gráficos
plt.tight_layout()
plt.show()

### Identificação e substituição de Outliers

Nesta etapa, implementamos uma função para identificar e substituir outliers em nosso conjunto de dados numéricos usando o método do intervalo interquartil (IQR). A função identificar_outliers_iqr calcula os quartis (Q1 e Q3) e determina o IQR, definindo limites inferior e superior para detectar valores atípicos.

Em seguida, aplicamos essa função ao DataFrame df_train_num para gerar um DataFrame booleano, onde cada valor indica se o respectivo elemento é um outlier.

A função substituir_outliers é responsável por substituir esses outliers pelo valor da mediana da coluna correspondente, ajudando a mitigar a influência negativa que os outliers podem ter no modelo preditivo. Esse processo é repetido até que não sejam encontrados mais outliers no DataFrame, garantindo que os dados estejam mais limpos e prontos para o treinamento do modelo. Essa abordagem é crucial para melhorar a robustez do modelo, evitando que ele seja influenciado por valores extremos que não representam bem a distribuição da maioria dos dados.

In [222]:

# Função para identificar os ouliers a partir do método IQR
def identificar_outliers_iqr(column):
    Q1 = column.quantile(0.25)
    Q3 = column.quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    return (column < limite_inferior) | (column > limite_superior)

# Aplica a função no dataframe numérico de treino
outliers = df_train_num.apply(identificar_outliers_iqr)

# Cria uma função para substituir os outliers
def substituir_outliers(df):
    outliers = df_train_num.apply(identificar_outliers_iqr)
    # Itera até que sejam substituidos todos outliers pela mediana
    while outliers.any().any() == True:
        for column in df.columns:
            no_outliers = df.loc[~outliers[column], column]
            median = no_outliers.median()
            df.loc[outliers[column], column] = median
        outliers = df.apply(identificar_outliers_iqr)
    
    return df

# Aplica a função de substituir no dataframe número de treino
substituir_outliers(df_train_num)

for coluna in df_train_num.columns:
    df_train[coluna] = df_train_num[coluna]


### Análise de Correlação

Nesta etapa, calculamos a matriz de correlação das variáveis numéricas no conjunto de treinamento utilizando o método corr() do pandas. A correlação é uma medida que indica a relação entre duas variáveis, variando de -1 a 1. Valores próximos a 1 indicam uma forte correlação positiva, enquanto valores próximos a -1 indicam uma forte correlação negativa. Valores próximos a 0 sugerem pouca ou nenhuma correlação.

Para visualizar essa matriz de correlação de forma clara e informativa, utilizamos um gráfico de calor (heatmap) da biblioteca Seaborn. O gráfico foi configurado para exibir os coeficientes de correlação dentro de cada célula, facilitando a interpretação dos dados. A paleta de cores 'coolwarm' foi utilizada para destacar visualmente as correlações positivas e negativas.

Essa análise é fundamental para entender como as variáveis se relacionam entre si e identificar quais features podem ser mais relevantes para o modelo preditivo, permitindo uma seleção de características mais informada e aumentando as chances de sucesso do modelo.

In [ ]:
# Calcular a matriz de correlação
correlacao = df_train_num.corr()

# Configura o tamanho da figura
plt.figure(figsize=(10, 8))

# Plotar a matriz de correlação
sns.heatmap(correlacao, annot=True, cmap='coolwarm', linewidths=0.5)

# Exibir o gráfico
plt.show()

### Codificação Inicial de Variáveis

Nesta etapa, substituímos as colunas numéricas do DataFrame original de treinamento (df_train) pelas colunas processadas que não contêm outliers, que foram armazenadas em df_train_num. Isso garante que estamos utilizando um conjunto de dados mais limpo para o desenvolvimento do modelo preditivo.

Além disso, realizamos a codificação da coluna explicit, que indica se uma música contém conteúdo explícito. Os valores 'FALSE' e 'TRUE' dessa coluna foram convertidos em valores numéricos, onde 'FALSE' foi substituído por 0 e 'TRUE' por 1. Esse tipo de transformação é essencial para que o modelo de aprendizado de máquina possa interpretar corretamente a variável categórica, já que a maioria dos algoritmos de machine learning, incluindo o Random Forest, trabalha melhor com dados numéricos.

O mesmo procedimento foi aplicado ao conjunto de teste (df_test) para garantir que ambas as bases de dados tenham o mesmo formato e estrutura, permitindo que o modelo treinado seja aplicado de forma consistente nas previsões. Essa etapa é crucial para a integridade dos dados e para a eficácia do modelo preditivo.

In [223]:
# Associa os outliers tratados na coluna original
for column in df_train_num:
    df_train[column] = df_train_num[column]

# Substitui os valores false e true para 0 e 1 respectivamente
df_train.loc[df_train['explicit'] == 'FALSE'] = 0
df_train.loc[df_train['explicit'] == 'TRUE'] = 1

df_test.loc[df_test['explicit'] == 'FALSE'] = 0
df_test.loc[df_test['explicit'] == 'TRUE'] = 1


### Preparações para a exploração e hipóteses
Nesta etapa, realizamos a filtragem do DataFrame de treinamento (df_train) para criar subconjuntos específicos de músicas baseados em seu gênero. Os gêneros selecionados foram gospel, MPB (Música Popular Brasileira) e J-Pop (música pop japonesa). Para cada gênero, utilizamos a função loc para extrair apenas as linhas correspondentes a cada categoria.

Após a filtragem, contamos a frequência dos valores da coluna popularity_target, que representa a popularidade das músicas dentro de cada gênero, utilizando a função value_counts(). Essa contagem nos fornece informações sobre quantas músicas possuem cada nível de popularidade (por exemplo, 0 ou 1).

Esta análise é parte do processo de preparação dos dados para a exploração das variáveis, permitindo-nos entender como a popularidade das músicas varia entre diferentes gêneros.

In [224]:
# Associa a uma variável, um dataframe onde armazena apenas as linhas onde a coluna 'track_genre' é igual ao genêro escolhido
gospel = df_train.loc[df_train['track_genre'] == 'gospel']
mpb = df_train.loc[df_train['track_genre'] == 'mpb']
j_pop = df_train.loc[df_train['track_genre'] == 'j-pop']

# Conta o número de valores achados por cada gênero
gospel_pop = gospel['popularity_target'].value_counts()
mpb_pop = mpb['popularity_target'].value_counts()
j_pop_pop = j_pop['popularity_target'].value_counts()


### Hipótese 1
Nesta etapa, formulamos uma hipótese que investiga a relação entre o gênero musical gospel e sua popularidade. Para isso, criamos um gráfico de pizza que ilustra a proporção de músicas gospel categorizadas como populares (1) e não populares (0).

A visualização, gerada a partir dos dados filtrados na variável gospel_pop, nos permite observar a distribuição de popularidade dentro desse gênero. Com isso, buscamos responder à pergunta: "As músicas gospel tendem a ser mais populares do que as não gospel?"

O gráfico de pizza apresenta as proporções de cada classe, facilitando a análise visual e a interpretação dos resultados.

In [ ]:
# Formula os gráficos para o gênero gospel e sua relação com a proporção de popularidade
labels = ['Popularidade 0', 'Popularidade 1']
colors = ['#ff9999','#66b3ff'][:len(gospel_pop)]


plt.figure(figsize=(6,6))
plt.pie(gospel_pop, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Proporção de Popularidade (0 ou 1) para Gospel')
plt.axis('equal')  # Equal aspect ratio para que o gráfico seja circular
plt.show()


### Hipótese 2
Nesta etapa, continuamos a formulação de hipóteses ao investigar a relação entre o gênero musical MPB (Música Popular Brasileira) e sua popularidade. Criamos um gráfico de pizza que ilustra a proporção de músicas MPB classificadas como populares (1) e não populares (0).

O gráfico, gerado a partir dos dados filtrados na variável mpb_pop, nos ajuda a observar como as músicas desse gênero estão distribuídas em termos de popularidade. Isso nos leva a questionar: "As músicas MPB têm uma taxa de popularidade similar ou diferente das músicas de outros gêneros?"

Ao visualizar essas proporções, buscamos identificar padrões que possam indicar se a MPB possui características únicas que influenciam sua popularidade. A interpretação clara desse gráfico é essencial para validar ou refutar nossa hipótese sobre a popularidade da música MPB.

In [ ]:
# Formula os gráficos para o gênero MPB e sua relação com a proporção de popularidade
labels = ['Popularidade 0', 'Popularidade 1']
colors = ['#ff9999','#66b3ff']


plt.figure(figsize=(6,6))
plt.pie(mpb_pop, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Proporção de Popularidade (0 ou 1) para MPB')
plt.axis('equal')  # Equal aspect ratio para que o gráfico seja circular
plt.show()

### Hipótese 3
Nesta etapa, continuamos a formulação de hipóteses ao analisar a relação entre o gênero musical J-Pop (Japanese Pop) e sua popularidade. Utilizando um gráfico de pizza, apresentamos a proporção de músicas J-Pop categorizadas como populares (1) e não populares (0).

O gráfico, gerado a partir dos dados filtrados na variável j_pop_pop, permite visualizar como as músicas desse gênero se distribuem em relação à popularidade. Isso nos leva a questionar: "As músicas J-Pop apresentam uma taxa de popularidade distinta em comparação com outros gêneros musicais?"

Ao observar essas proporções, buscamos identificar tendências que possam sugerir se o J-Pop possui atributos específicos que influenciam sua aceitação e sucesso entre os ouvintes. A interpretação deste gráfico é fundamental para validar ou refutar nossa hipótese sobre a popularidade da música J-Pop.

In [ ]:
# Formula os gráficos para o gênero J-Pop e sua relação com a proporção de popularidade
labels = ['Popularidade 0', 'Popularidade 1']
colors = ['#ff9999','#66b3ff']


plt.figure(figsize=(6,6))
plt.pie(j_pop_pop, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
plt.title('Proporção de Popularidade (0 ou 1) para J-Pop')
plt.axis('equal')  # Equal aspect ratio para que o gráfico seja circular
plt.show()

### Codificação de variáveis categóricas
Nesta etapa, aplicamos a codificação das variáveis categóricas, especificamente a coluna track_genre, que representa os gêneros musicais das faixas. Como os modelos de machine learning, como o Random Forest, requerem entradas numéricas, utilizamos o LabelEncoder do scikit-learn para transformar os gêneros musicais em valores numéricos.

O método fit_transform é aplicado ao conjunto de treinamento (df_train), onde cada gênero é mapeado para um número inteiro. Por exemplo, o gênero "gospel" pode ser codificado como 0, "mpb" como 1 e "j-pop" como 2, e assim por diante. Essa transformação é essencial para permitir que o modelo reconheça e utilize essas informações categóricas durante o treinamento.

Em seguida, realizamos a mesma transformação no conjunto de teste (df_test) para garantir que ambos os conjuntos de dados estejam em um formato consistente. A codificação das variáveis categóricas é uma etapa crítica na preparação dos dados, pois garante que as informações relevantes dos gêneros musicais sejam corretamente interpretadas pelo modelo, contribuindo para a precisão das previsões de popularidade musical.


In [225]:
# Codificando as features categóricas

le = LabelEncoder()
df_train['track_genre'] = le.fit_transform(df_train['track_genre'])
df_test['track_genre'] = le.fit_transform(df_test['track_genre'])


### Seleção de feature

Nesta etapa, realizamos a seleção de features, que é um processo essencial na construção de modelos preditivos. A seleção de features nos permite focar nas variáveis que mais impactam a previsão de nossa variável-alvo, neste caso, a popularidade da música.

Decidimos remover do conjunto de dados as colunas categóricas que não contribuiriam significativamente para a previsão do modelo, especificamente: track_id, artists, album_name, track_name e explicit. Embora essas variáveis possam conter informações valiosas, elas não são diretamente úteis para prever a popularidade das músicas, especialmente porque explicit já foi codificado como uma variável numérica.

O resultado dessa seleção é um conjunto de dados mais enxuto e focado, que contém apenas as features que se espera que tenham um impacto relevante na predição da popularidade. Essa abordagem não só ajuda a reduzir a complexidade do modelo, mas também pode melhorar o desempenho e a interpretabilidade do modelo ao evitar ruídos desnecessários nas variáveis. O mesmo processo de seleção de features foi aplicado ao conjunto de teste (df_test), assegurando consistência entre os dados utilizados para treinar e avaliar o modelo.

In [226]:
# Seleciona e exclui colunas para o treino do modelo
df_train = df_train.drop(columns=['track_id', 'artists', 'album_name', 'track_name', 'explicit'])

df_test = df_test.drop(columns=['track_id', 'artists', 'album_name', 'track_name', 'explicit'])


### Treinamento do modelo e avaliação do desempenho

Nesta etapa, realizamos o treinamento do modelo de machine learning utilizando o RandomForestClassifier. A Random Forest é uma técnica robusta que utiliza múltiplas árvores de decisão para fazer previsões, o que a torna eficaz em tarefas de classificação como a nossa, onde buscamos prever a popularidade de músicas.

Primeiramente, separamos os dados em variáveis independentes (X) e a variável dependente (y), que neste caso é a coluna popularity_target. Em seguida, dividimos o conjunto de dados em conjuntos de treinamento e teste utilizando a função train_test_split, reservando 30% dos dados para avaliação do modelo.

Após a separação dos dados, treinamos o modelo com o conjunto de treinamento (X_train e y_train) e, em seguida, realizamos previsões no conjunto de teste (X_test). A acurácia obtida foi de 82,11%, indicando que o modelo conseguiu prever corretamente a popularidade das músicas em mais de 82% dos casos testados.

Para uma avaliação mais detalhada, geramos uma matriz de confusão e um relatório de classificação, que fornecem insights sobre o desempenho do modelo em cada classe da variável-alvo. O relatório inclui métricas como precisão, recall e F1-score, que são fundamentais para entender a eficácia do modelo, especialmente em cenários de classificação desequilibrada.

A matriz de confusão visualiza os acertos e erros do modelo, permitindo identificar onde ele está se saindo bem e onde pode melhorar. Essas informações são cruciais para futuras iterações no modelo e para ajustes que possam aumentar sua performance.

In [ ]:
#Supondo que 'target' é a coluna que você deseja prever
X = df_train.drop(columns='popularity_target', axis=1) 
y = df_train['popularity_target']

# Features do conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Criar e treinar o modelo RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = rf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

print(f"Acurácia: {accuracy:.4f}\n")
print(f"Relatório de Classificação:\n{class_report}\n")
print(f"Matriz de Confusão:\n{conf_matrix}\n")


### Finetuning de Hiperparâmetros

Após o treinamento inicial do modelo, decidimos realizar um ajuste de hiperparâmetros para otimizar ainda mais seu desempenho. O ajuste de hiperparâmetros é uma etapa crucial no desenvolvimento de modelos de machine learning, pois permite encontrar a combinação ideal de parâmetros que maximiza a acurácia e melhora a generalização do modelo para novos dados.

Nesta fase, utilizamos a classe RandomizedSearchCV do Scikit-Learn, que nos permite explorar uma combinação de parâmetros de forma aleatória, economizando tempo em comparação com uma busca exaustiva. Especificamente, definimos um dicionário param_dist com os seguintes hiperparâmetros a serem ajustados:

- n_estimators: número de árvores na floresta (50 ou 100).
- max_depth: profundidade máxima das árvores (10 ou 30).
- min_samples_split: número mínimo de amostras necessárias para dividir um nó (2 ou 5).
- min_samples_leaf: número mínimo de amostras em um nó folha (1 ou 2).
- bootstrap: se as amostras devem ser extraídas com reposição (True ou False).

Com isso, criamos uma instância do RandomForestClassifier e aplicamos o RandomizedSearchCV, configurando 10 iterações e uma validação cruzada de 3 dobras. O modelo ajustado foi então avaliado no conjunto de teste.

A acurácia obtida após o ajuste foi avaliada e impressa, juntamente com um relatório de classificação e a matriz de confusão, que fornecem uma visão abrangente do desempenho do modelo após a otimização.

Por fim, exibimos os melhores hiperparâmetros encontrados durante o ajuste, que podem ser utilizados para futuros treinamentos ou como referência para ajustar outros modelos. Essa abordagem não apenas melhora a performance do modelo, mas também fornece insights valiosos sobre como os diferentes parâmetros afetam a previsão da popularidade das músicas.

In [ ]:
# Parâmetros a serem ajustados
param_dist = {
    'n_estimators': [50, 100],          
    'max_depth': [10, 30],           
    'min_samples_split': [2, 5],              
    'min_samples_leaf': [1, 2],                 
    'bootstrap': [True, False]                     
}

# Criar o modelo RandomForestClassifier
rf = RandomForestClassifier(random_state=42)

# RandomizedSearchCV
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=10, 
                               cv=3, verbose=2, random_state=42, n_jobs=-1)

# Ajustar o modelo
rf_random.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = rf_random.predict(X_test)

# Avaliação
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Exibir os resultados
print(f"Acurácia: {accuracy:.4f}\n")
print(f"Relatório de Classificação:\n{class_report}\n")
print(f"Matriz de Confusão:\n{conf_matrix}\n")

# Melhor modelo e hiperparâmetros
print(f"Melhores hiperparâmetros: {rf_random.best_params_}")

### Submissão dos resultados

Após a otimização do modelo com o ajuste de hiperparâmetros, a próxima etapa consiste em fazer previsões sobre os dados de teste, que não incluíam a coluna de popularidade em nosso DataFrame. Para isso, o conjunto X_test foi definido como df_test, o qual contém as mesmas características acústicas e metadados das músicas a serem analisadas.

O modelo ajustado, rf_random, foi então treinado novamente com os dados de treino X e y para garantir que estivesse completamente otimizado. Em seguida, utilizamos o método predict para gerar as previsões de popularidade para as músicas no conjunto de teste. As previsões foram armazenadas na variável test_pred.

Para preparar os dados para a submissão, criamos um novo DataFrame chamado submissao, que contém duas colunas: track_unique_id, que identifica de forma única cada faixa, e popularity_target, que armazena as previsões de popularidade geradas pelo modelo, convertidas para o tipo inteiro.

Por fim, para organizar os dados de forma adequada para a submissão no Kaggle, aplicamos uma operação de agrupamento, onde as entradas foram agrupadas pelo track_unique_id, garantindo que, em casos de duplicatas, apenas a previsão de popularidade mais alta fosse considerada. O resultado final foi então exportado para um arquivo CSV chamado submissao.csv, pronto para ser enviado como parte do projeto.

In [ ]:
# Separa um novo conjunto de dados para comportar uma previsão de uma coluna não incluida no dataframe
X_test = df_test

rf_random.fit(X, y)
test_pred = rf_random.predict(X_test)

# Ajusta as colunas do novo dataframe para submissão da atividade
submissao = pd.DataFrame({
    'track_unique_id': df_test['track_unique_id'],
    'popularity_target': test_pred.astype(int)
})

# Organiza e agrupa as colunas para a exporação em CSV
submissao_agrupada = submissao.groupby('track_unique_id', as_index=False).agg({'popularity_target': 'max'})
submissao_agrupada.to_csv('submissao.csv', index=False)
